In [2]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import numpy as np
import random
import cv2

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import torchvision.models as models

In [3]:
train_dir = '/opt/ml/input/data/train/'
train_image_path = '/opt/ml/input/data/train/images/'

test_dir = '/opt/ml/input/data/eval/'
test_image_path = '/opt/ml/input/data/eval/images/'

dt_train = pd.read_csv(train_dir+'train.csv')
dt_train

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59
...,...,...,...,...,...
2695,006954,male,Asian,19,006954_male_Asian_19
2696,006955,male,Asian,19,006955_male_Asian_19
2697,006956,male,Asian,19,006956_male_Asian_19
2698,006957,male,Asian,20,006957_male_Asian_20


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
random_seed = 12

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [6]:
# 일단은 list로 값 받는 형태로 _ Dataset 안에서 처리할 수도 있을듯
whole_image_path = []
whole_target_label = []

for path in dt_train['path']:
    for file_name in [i for i in os.listdir(train_image_path+path) if '._' not in i]:
        whole_image_path.append(train_image_path+path+'/'+file_name)
        whole_target_label.append((path.split('_')[1], path.split('_')[3], file_name.split('.')[0]))
        
        
# 라벨을 0~17로 할당하는 함수
def onehot_enc(x):
    # x 입력형태 : (gender, age, mask)의 튜플
    def gender(i):
        if i == 'male':
            return 0
        elif i == 'female':
            return 3
    def age(j):
        j = int(j)
        if j < 30:
            return 0
        elif j >= 30 and j < 60:
            return 1
        elif j >= 60:
            return 2
    def mask(k):
        if k == 'normal':
            return 12
        elif 'incorrect' in k:
            return 6
        else:
            return 0
    return gender(x[0]) + age(x[1]) + mask(x[2])

In [7]:
# sr_data : 이미지 데이터의 경로
# sr_label : 이미지 데이터의 라벨정보 (not_encoded)
sr_data = pd.Series(whole_image_path)
sr_label = pd.Series(whole_target_label)
#64, 447

In [8]:
class Dataset_Mask(Dataset):
    def __init__(self, encoding=True, midcrop=True, transform=None):
        self.encoding = encoding
        self.midcrop = midcrop
        self.data = sr_data
        self.label = sr_label
        self.transform = transform
        
        if encoding:
            self.label = self.label.apply(onehot_enc)
        
    def __len__(self):
        return len(sr_data)
    
    def __getitem__(self, idx):
        X = cv2.cvtColor(cv2.imread(self.data[idx]), cv2.COLOR_BGR2RGB)
        y = self.label[idx]
        
        if self.midcrop:
            X = X[64:448]
        
        if self.transform:
            return self.transform(X), y
        return X, y

In [9]:
dataset_mask = Dataset_Mask(transform = transforms.Compose([
                                transforms.ToTensor()
                            ]))

In [10]:
train_size = int(len(dataset_mask) * 0.8)
val_size = int(len(dataset_mask) * 0.2)

train_set, val_set = torch.utils.data.random_split(dataset_mask, [train_size, val_size])
print(f'training data size : {len(train_set)}')
print(f'validation data size : {len(val_set)}')

training data size : 15120
validation data size : 3780


In [11]:
batch_size = 64
train_dataLoader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True, num_workers=0)
val_dataLoader = DataLoader(dataset = val_set, batch_size=batch_size, shuffle=True, num_workers=0)

In [12]:
num_classes = 18
model_vgg19 = models.vgg19(pretrained=True)

print(model_vgg19)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [13]:
model_vgg19.classifier = nn.Sequential(
    nn.Linear(512 * 7 * 7, 4096),
    nn.ReLU(True),
    nn.Dropout(),
    nn.Linear(4096, 4096),
    nn.ReLU(True),
    nn.Dropout(),
    nn.Linear(4096, num_classes),
)

print(model_vgg19)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [14]:
import torch.nn.init as init

def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, nn.Conv2d):
            init.xavier_uniform_(m.weight.data)
            if m.bias is not None:
                m.bias.data.zero_()
        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()
        elif isinstance(m, nn.Linear):
            m.weight.data.normal_(0, 0.01)
            m.bias.data.zero_()

initialize_weights(model_vgg19.classifier)

In [15]:
epochs = 10
learning_rate = 0.0001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_vgg19.parameters(), lr=learning_rate, betas=(0.9,0.99))

In [16]:
model_vgg19.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [17]:
np.set_printoptions(precision=3)
n_param = 0
for p_idx, (param_name, param) in enumerate(model_vgg19.named_parameters()):
    if param.requires_grad:
        param_numpy = param.detach().cpu().numpy()
        n_param += len(param_numpy.reshape(-1))
        print ("[%d] name:[%s] shape:[%s]."%(p_idx,param_name,param_numpy.shape))
        print ("    val:%s"%(param_numpy.reshape(-1)[:5]))
print ("Total number of parameters:[%s]."%(format(n_param,',d')))

[0] name:[features.0.weight] shape:[(64, 3, 3, 3)].
    val:[-0.053 -0.049 -0.068  0.015  0.045]
[1] name:[features.0.bias] shape:[(64,)].
    val:[-0.913  0.307 -1.306 -0.776 -0.789]
[2] name:[features.2.weight] shape:[(64, 64, 3, 3)].
    val:[0.053 0.083 0.086 0.027 0.033]
[3] name:[features.2.bias] shape:[(64,)].
    val:[-0.058 -0.148  0.18  -0.286  0.014]
[4] name:[features.5.weight] shape:[(128, 64, 3, 3)].
    val:[-0.024  0.005  0.002 -0.033  0.091]
[5] name:[features.5.bias] shape:[(128,)].
    val:[0.025 0.133 0.008 0.05  0.031]
[6] name:[features.7.weight] shape:[(128, 128, 3, 3)].
    val:[ 0.015 -0.037  0.053 -0.002 -0.052]
[7] name:[features.7.bias] shape:[(128,)].
    val:[ 0.084 -0.082 -0.069 -0.016  0.437]
[8] name:[features.10.weight] shape:[(256, 128, 3, 3)].
    val:[ 0.036  0.05   0.051 -0.065 -0.061]
[9] name:[features.10.bias] shape:[(256,)].
    val:[-0.057  0.025  0.091 -0.321  0.012]
[10] name:[features.12.weight] shape:[(256, 256, 3, 3)].
    val:[ 0.022  0.

In [18]:
best_val_acc_so_far = 0
best_val_loss_so_far = np.inf

for epoch in range(epochs):
    print("[INFO] Training Epoch : {}".format(epoch))
    model_vgg19.train()

    for i, (X, y) in enumerate(train_dataLoader):
        loss_value = 0
        acc = 0
        
        X = X.to(device)
        y = y.to(device)
        
        _y = model_vgg19(X)
        _pred = torch.argmax(_y, dim=-1)

        loss = criterion(_y, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
                
        loss_value += loss.item()
        acc += (y == _pred).sum().item()

        train_loss = loss_value / batch_size
        train_acc = acc / batch_size

        if i % 10 == 0:
            print(f"STEP[{i}/{len(train_dataLoader)}] Training loss: {train_loss:.3f} / Training accuracy: {train_acc:.3f}")

    with torch.no_grad():
        model_vgg19.eval()

        val_loss_items = []
        val_acc_items = []

        for i, (X, y) in enumerate(val_dataLoader):
            X = X.to(device)
            y = y.to(device)
            
            _y = model_vgg19(X)
            _pred = torch.argmax(_y, dim=-1)
            
            loss_item = criterion(_y, y).item()
            acc_item = (y == _pred).sum().item()

            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)
            
        cur_val_loss = np.sum(val_loss_items) / len(val_dataLoader)
        cur_val_acc = np.sum(val_acc_items) / len(val_dataLoader)
        
        if cur_val_loss < best_val_loss_so_far:
            best_val_loss_so_far = cur_val_loss
        if cur_val_acc > best_val_acc_so_far:
            best_val_acc_so_far = cur_val_acc

        print(f"[Val] Current acc : {cur_val_acc:.3f} / Current loss : {cur_val_loss:.3f}")
        print(f"[Val] Best acc : {best_val_acc_so_far:.3f} / Best loss : {best_val_loss_so_far:.3f}")

    print(f"EPOCH[{epoch}/{epochs}] Training loss: {train_loss:.3f} / Training accuracy: {train_acc:.3f}")

print("[INFO] Training ALL DONE!!!")

[INFO] Training Epoch : 0
STEP[0/237] Training loss: 0.046 / Training accuracy: 0.047
STEP[10/237] Training loss: 0.029 / Training accuracy: 0.312
STEP[20/237] Training loss: 0.020 / Training accuracy: 0.609
STEP[30/237] Training loss: 0.018 / Training accuracy: 0.641
STEP[40/237] Training loss: 0.011 / Training accuracy: 0.781
STEP[50/237] Training loss: 0.011 / Training accuracy: 0.781
STEP[60/237] Training loss: 0.012 / Training accuracy: 0.781
STEP[70/237] Training loss: 0.011 / Training accuracy: 0.797
STEP[80/237] Training loss: 0.010 / Training accuracy: 0.766
STEP[90/237] Training loss: 0.007 / Training accuracy: 0.859
STEP[100/237] Training loss: 0.008 / Training accuracy: 0.828
STEP[110/237] Training loss: 0.008 / Training accuracy: 0.812
STEP[120/237] Training loss: 0.008 / Training accuracy: 0.875
STEP[130/237] Training loss: 0.007 / Training accuracy: 0.891
STEP[140/237] Training loss: 0.008 / Training accuracy: 0.812
STEP[150/237] Training loss: 0.007 / Training accuracy:

KeyboardInterrupt: 

In [19]:
submission = pd.read_csv(test_dir+'info.csv')
submission.head()

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,0
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,0
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,0
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,0
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0


In [20]:
image_paths = [os.path.join(test_image_path, img_id) for img_id in submission.ImageID]
test_image = pd.Series(image_paths)

In [21]:
class TestDataset(Dataset):
    def __init__(self, midcrop=True, transform=None):
        self.midcrop = midcrop
        self.data = test_image
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image = cv2.cvtColor(cv2.imread(self.data[idx]), cv2.COLOR_BGR2RGB)
        
        if self.midcrop:
            image = image[64:448]
        
        if self.transform:
            return self.transform(image)
        
        return image

In [22]:
test_transform = transforms.Compose([
                            transforms.ToTensor()
                        ])

test_set = TestDataset(transform = test_transform)
test_dataLodaer = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

In [25]:
model = model_vgg19.to(device)
model.eval()

all_predictions = []
for images in test_dataLodaer:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

test inference is done!
